# Browser-Use SDK を使用した基本的な Browser ツールの使い方

## 概要

このチュートリアルでは、オープンソースの Browser-Use SDK を使用して Amazon Bedrock AgentCore Browser ツールを操作する方法を学びます。ヘッドレスでの Browser ツールの使用例とブラウザのライブビュー表示の例を紹介します。


### チュートリアル詳細


| 項目         | 詳細                                                                                   |
|:--------------------|:-------------------------------------------------------------------------------------------        
| チュートリアルタイプ       | 会話型                                                                            |
| エージェントタイプ          | シングル                                                                                    |
| エージェントフレームワーク   | Browser-Use                                                                               |
| LLM モデル           | Anthropic Claude 3.7 Sonnet                                                               |
| チュートリアルコンポーネント | Browser-Use SDK を使用したヘッドレスでの Bedrock AgentCore Browser ツール操作   |
| チュートリアル分野   | クロスバーティカル                                                                            |
| 難易度  | 簡単                                                                                      |
| 使用 SDK            | Amazon Bedrock AgentCore Python SDK、Browser-Use                                           |

### チュートリアルアーキテクチャ

このチュートリアルでは、Browser-Use SDK と AgentCore Browser ツールの使用方法を説明します。  

この例では、Browser-Use エージェントに自然言語の指示を送り、Bedrock AgentCore Browser 上でヘッドレスでタスクを実行します。


### チュートリアルの主な機能

* ヘッドレスでの Browser ツールの使用
* Browser-Use と Browser ツールの連携

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.11+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Browser-Use SDK 

## 動作の仕組み

Browser ツールサンドボックスは、AI エージェントが Web ブラウザと安全に対話できるセキュアな実行環境です。ユーザーがリクエストを行うと、Large Language Model（LLM）が適切なツールを選択してコマンドを変換します。これらのコマンドは、ヘッドレスブラウザとホステッドライブラリサーバー（Playwright などのツールを使用）を含む制御されたサンドボックス環境内で実行されます。サンドボックスは、Web 操作を制限された空間内に閉じ込め、不正なシステムアクセスを防ぐことで、分離とセキュリティを提供します。エージェントはスクリーンショットを通じてフィードバックを受け取り、システムセキュリティを維持しながら自動化タスクを実行できます。 

![architecture local](../images/browser-tool.png)

## 1. 環境のセットアップ

#### 認証ヘッダーの問題を修正し、Amazon Bedrock AgentCore Browser と互換性を持たせるために、以下のスクリプトを実行してください。

### 1.1 必要なライブラリのインストール 

まず、Browser ツールサンドボックスクライアントに必要なライブラリをインストールしましょう。 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

### 1.2 browser-use 用パッチスクリプト

In [ ]:
%%writefile patch_browser_use.py

#!/usr/bin/env python3
"""Auto-detect and patch browser_use session.py"""

import os
import shutil
import sys
from pathlib import Path

def find_browser_use_path():
    """Automatically find the browser_use installation path"""
    try:
        import browser_use
        browser_use_path = Path(browser_use.__file__).parent
        session_file = browser_use_path / "browser" / "session.py"
        return str(session_file)
    except ImportError:
        print("❌ browser_use not installed. Install with: pip install browser-use")
        return None

def patch_browser_use():
    # Auto-detect file path
    file_path = find_browser_use_path()
    if not file_path:
        return False
    
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return False
    
    print(f"📁 Found browser_use at: {file_path}")
    
    # Create backup
    backup_path = file_path + ".backup"
    if not os.path.exists(backup_path):
        shutil.copy2(file_path, backup_path)
        print(f"💾 Created backup: {backup_path}")
    else:
        print(f"📋 Backup already exists: {backup_path}")
    
    # Read file
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Replacement 1: Add headers check after cdp_url check
    old1 = "if not cdp_url:\n\t\t\tprofile_kwargs['is_local'] = True"
    new1 = "if not cdp_url:\n\t\t\tprofile_kwargs['is_local'] = True\n\n\t\tif headers:\n\t\t\tprofile_kwargs['headers'] = headers"
    
    if old1 in content and "if headers:\n\t\t\tprofile_kwargs['headers'] = headers" not in content:
        content = content.replace(old1, new1)
        print("✅ Added headers check")
    elif "if headers:\n\t\t\tprofile_kwargs['headers'] = headers" in content:
        print("✅ Headers check already exists")
    else:
        print("⚠️ Headers check pattern not found")
    
    # Replacement 2: Add headers to CDPClient
    old2 = "self._cdp_client_root = CDPClient(self.cdp_url)"
    new2 = "self._cdp_client_root = CDPClient(self.cdp_url,  additional_headers=self.browser_profile.headers)"
    
    if old2 in content:
        content = content.replace(old2, new2)
        print("✅ Added headers to CDPClient")
    elif "additional_headers=self.browser_profile.headers" in content:
        print("✅ CDPClient headers already exists")
    else:
        print("⚠️ CDPClient pattern not found")
    
    # Write back
    with open(file_path, 'w') as f:
        f.write(content)
    
    print("🎉 Patching complete!")
    return True

if __name__ == "__main__":
    success = patch_browser_use()
    sys.exit(0 if success else 1)

In [ ]:
# Run the python script to apply the patch for browser-use
!python patch_browser_use.py

In [ ]:
# Restart the Kernal 
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

### 1.3 インポート 

Browser ツールサンドボックスクライアントを初期化するために必要なライブラリをインポートします。 

In [ ]:
from bedrock_agentcore.tools.browser_client import BrowserClient
from browser_use.llm import ChatAnthropicBedrock, ChatAWSBedrock
from browser_use import Agent
from browser_use import Browser, BrowserProfile
from rich.console import Console
from contextlib import suppress
import asyncio

In [ ]:
console = Console()

## 2. ブラウザクライアントのセットアップ
ブラウザクライアントをセットアップし、準備が整うまで待機します。その後、WebSocket URL とヘッダーを生成します。
 


In [ ]:
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

client = BrowserClient(region)
client.start()

# Extract ws_url and headers
ws_url, headers = client.generate_ws_headers()

## 4. ブラウザタスク実行用ヘルパー関数
browser-use Agent を使用してブラウザ自動化タスクを実行します。




In [ ]:
async def run_browser_task(browser_session: Browser, bedrock_chat: ChatAnthropicBedrock, task: str) -> None:
    """
    Run a browser automation task using browser_use
    
    Args:
        browser_session: Existing browser session to reuse
        bedrock_chat: Bedrock chat model instance
        task: Natural language task for the agent
    """
    try:
        # Show task execution
        console.print(f"\n[bold blue]🤖 Executing task:[/bold blue] {task}")
        
        # Create and run the agent
        agent = Agent(
            task=task,
            llm=bedrock_chat,
            browser_session=browser_session
        )
        
        # Run with progress indicator
        with console.status("[bold green]Running browser automation...[/bold green]", spinner="dots"):
            await agent.run()
        
        console.print("[bold green]✅ Task completed successfully![/bold green]")
        
    except Exception as e:
        console.print(f"[bold red]❌ Error during task execution:[/bold red] {str(e)}")
        import traceback
        if console.is_terminal:
            traceback.print_exc()

## 5. Browser-use プロファイルを使用してブラウザタスクを呼び出し
CDP WebSocket 接続を使用して永続的なブラウザセッションを作成し、自動化 Web タスク用に Bedrock Claude モデルを初期化します。適切なクリーンアップでセッションのライフサイクルを処理し、AI 駆動のコマンドを通じてブラウザ自動化タスクを実行します。

In [ ]:
# Create persistent browser session and model
browser_session = None
bedrock_chat = None

try:
     # Create browser profile with headers and timeout 
    browser_profile = BrowserProfile(
        headers=headers,
        timeout=1500000,  # 150 seconds timeout
    )

    # Create a browser session with CDP URL and keep_alive=True for persistence
    browser_session = Browser(
        cdp_url=ws_url,
        browser_profile=browser_profile,
        keep_alive=True
    )
    
    # Initialize the browser session
    console.print("[cyan]🔄 Initializing browser session...[/cyan]")
    await browser_session.start()
    
    # Create ChatBedrockConverse once
    bedrock_chat = ChatAnthropicBedrock(
		model='global.anthropic.claude-haiku-4-5-20251001-v1:0',
		aws_region='us-west-2'
	)
    
    console.print("[green]✅ Browser session initialized and ready for tasks[/green]\n")

    task = "Search for a coffee maker on amazon.com and extract details of the first one" ## Modify the task to run other tasks

    await run_browser_task(browser_session, bedrock_chat, task)

finally:
    # Close the browser session
    if browser_session:
        console.print("\n[yellow]🔌 Closing browser session...[/yellow]")
        with suppress(Exception):
            await browser_session.close()
        console.print("[green]✅ Browser session closed[/green]")


## 6. クリーンアップ
まだ停止していない場合は、ブラウザセッションを停止します。

In [ ]:
client.stop()
print("ブラウザセッションが正常に停止しました！")